In [1]:
!pip install faiss-cpu
!pip install gensim
!pip install pybind11>2.12
!pip install numpy==1.26.4

In [15]:
import faiss # chaining vectors into one meaningful one
from gensim.models import Word2Vec

import nltk # to convert text into some more presentable way
nltk.download("punkt")
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [24]:
def sentenses_convert(file_name):
  with open(file_name, "r", encoding="utf-8") as file: ## reading a file and storing it into one variable
    f = file.read()

  sentenses = sent_tokenize(f)
  tokenized_sentenses = [word_tokenize(sent) for sent in sentenses] # structure a text

  model = Word2Vec(tokenized_sentenses, vector_size=100, window=5, min_count=2, sg=1) # convertation to vectors
  return model.wv.index_to_key


In [25]:
sentenses_convert("church_text")

['the',
 ',',
 'of',
 'and',
 '.',
 'in',
 'to',
 'is',
 'its',
 'The',
 'a',
 'church',
 'it',
 'with',
 'as',
 'mosaic',
 'monument',
 'century',
 'also',
 'wall',
 '(',
 'this',
 'which',
 ')',
 'at',
 'on',
 'art',
 'artistic',
 'Christian',
 'Aggeloktisti',
 'for',
 'from',
 'that',
 'Panagia',
 'by',
 'religious',
 'different',
 'chapel',
 'built',
 'period',
 'within',
 'Heritage',
 'was',
 'World',
 'mosaics',
 'works',
 'still',
 'early',
 'conservation',
 'an',
 'cultural',
 'rural',
 'development',
 'has',
 'are',
 'values',
 'Universal',
 'will',
 'Byzantine',
 'Value',
 '11th',
 'Outstanding',
 'It',
 'Antiquities',
 'Criterion',
 ':',
 'Conservation',
 'Virgin',
 'significance',
 'Mediterranean',
 'developed',
 'Cyprus',
 'antiquity',
 'monuments',
 'original',
 'diachronic',
 'importance',
 'As',
 'represents',
 'be',
 'These',
 'most',
 'significant',
 'List',
 'artistry',
 'same',
 'including',
 'over',
 'authenticity',
 'community',
 'In',
 '12th',
 'traditions',
 'we